# Load env and files

In [20]:
import os
os.chdir('..')

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from fossil_classification import *
from enrich_holdings import *
from reports_etl import *
from holdings_analysis import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.3f}".format

In [4]:
response_path = "data/downloaded reports/company reports 2021Q3/"
all_holdings_cls = pd.read_csv(response_path+"all_holdings_cls_all_ids.csv", dtype=holdings_dtypes())
# all_holdings_cls = load_mappings_and_add_ids_to_holdings(all_holdings_cls)

/Users/urimarom/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,7,13,20,22,23,24,25,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Let the Analysis begin!

In [5]:
pd.DataFrame(all_holdings_cls.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"2,632,863,710.94"
גמל,"3,890,353,769.96"
פנסיה,"3,622,905,980.61"


## Get Company level stats over time

### Filter results to include only major companies

In [6]:
all_holdings_cls_filtered = filter_major_companies(all_holdings_cls)

### * should include other subsidiary funds as well?

In [7]:
included_comp_group = all_holdings_cls_filtered["ParentCorpGroup"].unique()
included_comp = all_holdings_cls_filtered["ParentCorpName"].unique()
all_comp = all_holdings_cls["ParentCorpName"].unique()
not_included_comp = [c for c in all_comp if c not in included_comp]
print([c for c in not_included_comp if any(g in re.split('-| ',c) for g in included_comp_group)])

pd.DataFrame(
    all_holdings_cls[
    all_holdings_cls["ParentCorpName"].isin(not_included_comp)
    ].groupby(["ParentCorpName", "SystemName"])["שווי"].sum()
).sort_values("שווי", ascending=False)

['הלמן-אלדובי חח"י גמל בע"מ', 'מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל בע"מ']


,,שווי
ParentCorpName,SystemName,
"הלמן-אלדובי חח""י גמל בע""מ",פנסיה,"294,555,428.78"
"קרנות השתלמות למורים ולגננות - חברה מנהלת בע""מ",גמל,"130,094,753.26"
"אנליסט קופות גמל בע""מ",גמל,"70,437,523.83"
"הכשרה חברה לביטוח בע""מ",ביטוח,"68,124,180.25"
"קרנות השתלמות למורים תיכוניים, מורי סמינרים ומפקחים - חברה מנהלת בע""מ",גמל,"47,970,533.36"
"עגור חברה לניהול קופות גמל וקרנות השתלמות בע""מ",גמל,"42,584,010.05"
"קרן החסכון לצבא הקבע - חברה לניהול קופות גמל בע""מ",גמל,"39,630,960.06"
"איילון חברה לביטוח בע""מ",ביטוח,"31,025,253.64"
"גל -ניהול קופות גמל לעובדי הוראה בע""מ",גמל,"30,198,541.47"


In [8]:
company_stats_since_2020 = get_summary(all_holdings_cls_filtered, 'ReportPeriodDate', 'ParentCorpGroup')
company_stats_since_2020 = company_stats_since_2020.sort_values(
    ["ReportPeriodDate", "ParentCorpGroup"], ascending=(False, True)
)

In [9]:
# company_stats_since_2020[company_stats_since_2020['ParentCorpGroup'] == 'מגדל']

In [10]:
company_stats_since_2020.to_csv(response_path+"company_stats_since_2020.csv", index=False)

In [11]:
company_system_stats_since_2020 = get_summary(
    all_holdings_cls_filtered,
    'ReportPeriodDate', 'ParentCorpGroup', 'SystemName'
)

In [12]:
company_system_stats_since_2020.to_csv(response_path+"company_system_stats_since_2020.csv", index=False)

In [13]:
company_system_holding_type_stats_since_2020 = get_summary(
    all_holdings_cls_filtered,
    'ReportPeriodDate', 'ParentCorpGroup', 'SystemName', 'holding_type'
)

In [14]:
company_system_holding_type_stats_since_2020.to_csv(
    response_path+"company_system_holding_type_stats_since_2020.csv", index=False)

### Export aggregated data for quarterly org ranking

In [6]:
midrag_agg = get_latest_q_ranking_agg_from_holdings(filter_major_companies(all_holdings_cls))

In [7]:
midrag_agg.to_csv(
    response_path+"midrag_agg_latest_q.csv", index=False
)

Upload results to G-Drive: https://drive.google.com/drive/u/2/folders/1n0F9JpyJ4LdWxLc4-a8XaRaMV20qo1Fx

## Mergers - update
### 1. update holdings data according to system mergers

In [15]:
majors_after_mergers = filter_major_companies(all_holdings_cls)
# including subsidiaries
# majors_after_mergers = filter_major_companies(all_holdings_cls, include_subsidiaries=True)

# leave only latest quarter
majors_after_mergers = majors_after_mergers[
    majors_after_mergers["ReportPeriodDate"] == majors_after_mergers["ReportPeriodDate"].max()
]

# change data according to mergers
majors_after_mergers = update_system_owners(majors_after_mergers, "גמל", "הלמן", "הפניקס")
majors_after_mergers = update_system_owners(majors_after_mergers, "פנסיה", "הלמן", "מיטב")
majors_after_mergers = update_system_owners(majors_after_mergers, "גמל", "פסגות", "אלטשולר")
majors_after_mergers = update_system_owners(majors_after_mergers, "פנסיה", "פסגות", "הראל")

moving 15,677,553.290000001 from הלמן גמל to הפניקס גמל
moving 0.0 from הלמן פנסיה to מיטב פנסיה
moving 49,780,952.0 from פסגות גמל to אלטשולר גמל
moving 3,964,347.4000000004 from פסגות פנסיה to הראל פנסיה


In [27]:
company_system_stats_after_mergers = get_summary(
    majors_after_mergers,
    'ReportPeriodDate', 'ParentCorpGroup'
)
# company_system_stats_after_mergers.sort_values("שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים")
company_system_stats_after_mergers.sort_values("שיעור פוסילי במניות ואגח קונצרני סחירים")[[
    "ReportPeriodDate",
    "ParentCorpGroup",
    "שווי",
    "שווי במניות ואגח קונצרני סחירים",
    "שיעור פוסילי במניות ואגח קונצרני סחירים"
]]

# company_system_stats_after_mergers["ratio"] = (
#     company_system_stats_after_mergers["שיעור פוסילי במניות ואגח קונצרני סחירים"] / 
#     company_system_stats_after_mergers["שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים"]
# )

# company_system_stats_after_mergers.sort_values("ratio")

,ReportPeriodDate,ParentCorpGroup,שווי,שווי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים
1,2021-09-30,אלטשולר,"244,289,299.667","76,342,208.937",0.051
8,2021-09-30,מיטב,"65,277,271.886","24,158,188.530",0.091
5,2021-09-30,כלל,"219,300,212.371","83,304,891.447",0.100
6,2021-09-30,מגדל,"255,819,790.717","88,004,560.416",0.106
3,2021-09-30,הראל,"212,810,413.962","53,678,948.954",0.109
2,2021-09-30,הפניקס,"176,565,517.319","63,878,498.050",0.125
7,2021-09-30,מור,"23,969,445.784","8,230,452.765",0.127
9,2021-09-30,מנורה,"242,050,749.128","69,279,434.672",0.130
4,2021-09-30,ילין,"44,916,636.221","13,570,243.313",0.178
0,2021-09-30,אינפיניטי,"1,409,265.790","292,361.080",0.179


### Export aggregated data <u> after mergers </u> for quarterly org ranking

In [12]:
midrag_agg_after_mergers = get_latest_q_ranking_agg_from_holdings(majors_after_mergers)

In [13]:
midrag_agg_after_mergers.to_csv(
    response_path+"midrag_after_mergers_with_subsidiaries.csv", index=False
)

# Export holdings data for quarterly ranking

In [53]:
q = '2021 רבעון 3'
holding_types_for_ranking = ['מניות', 'אג"ח קונצרני']
cols = [
    'שם המנפיק/שם נייר ערך',
    'מספר ני"ע',
    'ISIN',
    'מספר מנפיק',
    'מספר תאגיד',
    'LEI',
    'ענף מסחר',
    'שווי',
    'שעור מנכסי אפיק ההשקעה',
    'שעור מסך נכסי השקעה',
    'שעור מערך נקוב מונפק',
    'holding_type',
    'SystemName',
    'ParentCorpGroup',
    'is_fossil',
    'שווי פוסילי'
]
holdings_for_ranking = filter_major_companies(all_holdings_cls)
holdings_for_ranking = holdings_for_ranking.loc[
    (holdings_for_ranking['ReportPeriodDesc'] == q) &
    (holdings_for_ranking["holding_type"].isin(holding_types_for_ranking)),
    cols
]
# use ISIN when Israeli sec num is not available
holdings_for_ranking['מספר ני"ע'] = holdings_for_ranking['מספר ני"ע'].fillna(holdings_for_ranking['ISIN'])
# use Israeli corp num and then LEI when issuer number is not available
holdings_for_ranking['מספר מנפיק'] = holdings_for_ranking['מספר מנפיק'].fillna(holdings_for_ranking['מספר תאגיד'])
holdings_for_ranking['מספר מנפיק'] = holdings_for_ranking['מספר מנפיק'].fillna(holdings_for_ranking['LEI'])
holdings_for_ranking.drop(['ISIN', 'מספר תאגיד', 'LEI'], axis=1, inplace=True)
holdings_for_ranking.rename({
    "holding_type": "סוג החזקה",
    "SystemName":  "אפיק",
    "ParentCorpGroup": "גוף"
}, axis=1, inplace=True)

# holdings_for_ranking[
#     (holdings_for_ranking["is_fossil"] == 1) &
#     ~(holdings_for_ranking["שווי פוסילי"] > 0)
# ]

holdings_for_ranking.to_csv(response_path+"quarterly_ranking_data.csv", index=False)

## Export csv per institution

In [68]:
inst_cols = [c for c in holdings_for_ranking.columns if (c != 'גוף') & ~(c.startswith("שעור"))]
for inst in get_major_institutions_list():
    holding_per_inst = holdings_for_ranking.loc[
        (holdings_for_ranking["גוף"] == inst),
        inst_cols
    ].sort_values("שווי פוסילי", ascending=False)
    holding_per_inst.to_csv(response_path+inst+".csv", index=False)

# Get a list of all fossil holdings for major institutions for a given quarter

In [32]:
q = ['2021 רבעון 3']
inst = get_major_institutions_list()
fossil_holdings_last_q = group_fossil_holdings_quarters_institutions(
    all_holdings_cls_filtered,
    quarters=q,
    institutions = inst
)

fossil_holdings_last_q.to_csv("analysis/last_q_major_insts_fossil_holdings.csv", index=False)

total Israeli fossil holdings: 38536148.68233508
total fossil holdings with il_sec_num: 38536148.68233508
total fossil holdings without il_sec_num: 11535236.331960699
11535236.331960699
after adding Israeli and non-Israeli: 50071385.01429577


# Compare Fossil Holdings over quarters
## 1. for specific companies

In [66]:
company_for_comparison = ['ילין']
quarters_for_comparison = [
    '2021 רבעון 2',
    '2021 רבעון 3'
]
comparison = compare_fossil_holdings_over_quarters(all_holdings_cls_filtered, quarters_for_comparison, company_for_comparison)
comparison.sort_values("name", ascending=False)

total Israeli fossil holdings: 4593530.859722018
total fossil holdings with il_sec_num: 4593530.859722019
total fossil holdings without il_sec_num: 7105.15618
7105.15618
after adding Israeli and non-Israeli: 4600636.015902018


,name,id,fossil_sum_prev_q,quantity_sum_prev_q,fossil_sum_curr_q,quantity_sum_curr_q,quantity_diff,quantity_diff_pct,fossil_sum_diff,fossil_sum_diff_pct
26,תמר פטרוליום,1689,"168,965.72","181,954,598.40","163,151.24","174,033,900.48","-7,920,697.92",-0.04,"-5,814.49",-0.03
25,שמן נפט וגז,514532456,0.00,0.90,0.00,0.00,-0.90,-1.00,-0.00,-1.00
24,שיכון ובינוי,1068,"58,717.47","26,318,137.34","43,632.91","15,039,374.34","-11,278,763.00",-0.43,"-15,084.55",-0.26
31,רציו מימון,394,0.00,0.00,"2,377.82","2,370,000.00","2,370,000.00",inf,"2,377.82",inf
23,רפק אגח ו,769,"38,563.43","12,179,989.00","36,649.79","12,454,636.00","274,647.00",0.02,"-1,913.64",-0.05
30,רימון מועמדת,1928,0.00,0.00,"5,136.50","155,935.00","155,935.00",inf,"5,136.50",inf
22,קנון,1635,"98,181.40","873,500.00","119,293.82","892,250.00","18,750.00",0.02,"21,112.43",0.22
21,פטרוכימיים,520029315,"12,058.78","51,221,781.00","10,289.72","51,221,781.00",0.00,0.00,"-1,769.06",-0.15
20,פז נפט,1363,"70,435.20","176,000.00","66,300.04","188,889.00","12,889.00",0.07,"-4,135.16",-0.06
19,סופרגז,1804,"42,168.39","14,392,568.00","41,747.84","14,392,866.00",298.00,0.00,-420.55,-0.01


In [67]:
company_quarter = company_for_comparison[0] + " - " + quarters_for_comparison[1]
comparison.to_csv("analysis/fossil holdings comparison - " + company_quarter + ".csv", index=False)

## 2. for all (major) companies

In [73]:
all_holdings_cls_filtered_grouped = all_holdings_cls_filtered.copy()
all_holdings_cls_filtered_grouped["ParentCorpGroup"] = "כל החברות"
all_major_companies = ["כל החברות"]
quarters_for_comparison = [
    '2021 רבעון 2',
    '2021 רבעון 3'
]

compare_all = compare_fossil_holdings_over_quarters(
    all_holdings_cls_filtered_grouped,
    quarters_for_comparison,
    all_major_companies
)

total Israeli fossil holdings: 76144868.64174868
total fossil holdings with il_sec_num: 76144868.6417487
total fossil holdings without il_sec_num: 24143102.99176176
24143102.99176176
after adding Israeli and non-Israeli: 100287971.63351044


In [74]:
quarters = ' '.join(quarters_for_comparison)
compare_all.to_csv("analysis/all companies - fossil holdings comparison - " + quarters + ".csv", index=False)

## Look into non fossil holding types

In [53]:
all_holdings_cls_non_fossil_types = all_holdings_cls_filtered[
    all_holdings_cls_filtered["holding_type"].isin(get_non_fossil_holding_types())
]

all_holdings_cls_non_fossil_types['holding_type'].unique()

array(['מזומנים', 'תעודות התחייבות ממשלתיות',
       'לא סחיר - תעודות התחייבות ממשלתי', 'פקדונות מעל 3 חודשים'],
      dtype=object)

In [54]:
all_holdings_cls_non_fossil_types.groupby("holding_type").agg({
      "שווי": "sum",
    "שווי פוסילי": "sum"
})

,שווי,שווי פוסילי
holding_type,,
לא סחיר - תעודות התחייבות ממשלתי,"1,040,434,526.80",0.00
מזומנים,"738,522,129.92",0.00
פקדונות מעל 3 חודשים,"53,212,857.20",0.00
תעודות התחייבות ממשלתיות,"1,491,058,879.74","29,677.35"


In [55]:
# holdings that should be non fossil by holding type but are fossil
all_holdings_cls_non_fossil_types[
    all_holdings_cls_non_fossil_types["is_fossil"] == 1
]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,ParentCorpGroup,ReportPeriodDate
13173,ISRELE 4.25 14/8/28,NaN,NaN,Baa2,Moodys,דולר אמריקאי,0.04,0.03,"14,919.64",0.00,0.00,2049749,תעודות התחייבות ממשלתיות,NYSE,NaN,6.34,"4,000,000.00",111.87,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"מור קופות גמל בע""מ",I_514956465,0.0,nan,nan,27/05/2021,2021 רבעון 1,1.00,"14,919.64",IL0060002446,NaN,NaN,מור,2021-03-31
20932,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,0.05,0.08,"2,366.65",0.01,0.00,1847126,תעודות התחייבות ממשלתיות,אחר,NaN,6.89,"576,000.00",115.25,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,08/07/2020,2020 רבעון 1,1.00,"2,366.65",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2020-03-31
20936,PEMEX 0 03/11/2,NaN,NaN,NR,NaN,דולר אמריקאי,0.06,0.02,517.37,0.00,0.00,1847126,תעודות התחייבות ממשלתיות,אחר,NaN,0.19,"176,000.00",82.46,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,08/07/2020,2020 רבעון 1,1.00,517.37,US71654QCF72,NaN,NaN,פסגות,2020-03-31
102585,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,0.05,0.04,"2,183.39",0.01,0.00,1945468,תעודות התחייבות ממשלתיות,אחר,NaN,7.60,"576,000.00",110.16,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,30/11/2020,2020 רבעון 3,1.00,"2,183.39",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2020-09-30
102588,PEMEX 0 03/11/2,NaN,NaN,NR,NaN,דולר אמריקאי,0.06,0.05,600.23,0.00,0.00,1945468,תעודות התחייבות ממשלתיות,אחר,NaN,0.20,"176,000.00",99.11,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,30/11/2020,2020 רבעון 3,1.00,600.23,US71654QCF72,NaN,NaN,פסגות,2020-09-30
133682,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,"2,204.06",0.01,0.00,1884615,תעודות התחייבות ממשלתיות,אחר,NaN,nan,"576,000.00",110.40,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/09/2020,2020 רבעון 2,1.00,"2,204.06",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2020-06-30
133686,PEMEX 0 03/11/2,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,592.68,0.00,0.00,1884615,תעודות התחייבות ממשלתיות,אחר,NaN,nan,"176,000.00",97.16,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/09/2020,2020 רבעון 2,1.00,592.68,US71654QCF72,NaN,NaN,פסגות,2020-06-30
146633,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,0.00,"2,331.49",0.01,0.00,2062624,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"599,000.00",116.75,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,20/06/2021,2021 רבעון 1,1.00,"2,331.49",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2021-03-31
146636,PEMEX 0 03/11/2,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,0.00,859.55,0.00,0.00,2062624,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"254,000.00",101.50,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,20/06/2021,2021 רבעון 1,1.00,859.55,US71654QCF72,NaN,NaN,פסגות,2021-03-31
403299,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,"2,273.40",0.01,0.00,2018568,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"599,000.00",118.05,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,05/04/2021,2020 רבעון 4,1.00,"2,273.40",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2020-12-31


## Export holdings with missing classification that should be classified

In [9]:
missing_cls = all_holdings_cls[
    (all_holdings_cls["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls["is_fossil"].isnull())
]

cols = ['שם המנפיק/שם נייר ערך' ,'מספר ני"ע' ,'מספר מנפיק', 'ISIN', 'מספר תאגיד', 'LEI']
missing_cls_deduped = missing_cls[cols].drop_duplicates()
print(missing_cls_deduped.shape[0])
missing_cls_deduped

13


,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,ISIN,מספר תאגיד,LEI
28731,CHKP US Equity- צ'ק פוינט,IL0010824113 EQUITY,NaN,NaN,520042821,NaN
38221,PLURISTEM Therapeutics Inc SYS- PLURISTEM THER...,US72940P1066-71013122,NaN,NaN,NaN,NaN
52733,PEMEX 5.95 28/01/31,USP71654QDE98,NaN,NaN,NaN,NaN
56742,GLAXOSMITHKLINE PLC- GLAXOSMITHKLINE PLC,US37733W1053-110637592,NaN,NaN,NaN,NaN
94603,Tencent holdings- Tencent holdings ltd,KY6875721220-60175411,NaN,NaN,NaN,NaN
114477,איי.אפ.אפ- אינטרנשיונל פליוורס אנד פראגרנסס אינק,US4595061015_1155019,1760,NaN,NaN,NaN
122042,Capital One- capital one,US14040H1059-60004140,NaN,NaN,NaN,NaN
141181,P-ADR,70422225,NaN,NaN,NaN,NaN
181606,LVS VEGAS SANDS CORP- LAS VEGAS SANDS CORP,71032965 - US5178341070,NaN,NaN,NaN,NaN
292935,CYBR 0 11/15/2024,US23248VAA35 C,NaN,NaN,NaN,NaN


In [15]:
missing_cls_deduped.to_csv(response_path+"missing_cls.csv", index=False)

# Simulation - change fossil classification for selected holdings, analyze results

In [17]:
# fix data in all_holdings_cls - wrong data for סופרגז
# all_holdings_cls[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז']

In [49]:
all_holdings_cls.loc[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז',
                    'מספר ני"ע'] = '1166917'

all_holdings_cls.loc[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז',
                     'ISIN'] = 'IL0011669178'

In [50]:
all_holdings_cls["clean_name"] = all_holdings_cls["שם המנפיק/שם נייר ערך"].apply(clean_company)

In [68]:
selected_holdings_names = ["כיל", "ICL", 'איי סי אל', 'איי.סי.אל', 'אייסיאל', 'ISRAEL CHEMICAL', 'כימיקלים לישראל', 'כי"ל',
                           "רימון",
                           "סופרגז", "סופר גז",
                           "OPC", 'אופיסי', 'או. פי. סי', 'או פי סי',
                           'חברה לישראל', 'החברה לישראל', 'חברהלישראל','חברה ישראל']
name_mask = all_holdings_cls["clean_name"].str.upper().str.startswith(tuple(selected_holdings_names))
all_holdings_cls.loc[name_mask].groupby(['מספר מנפיק'], dropna=False).agg(
    name=pd.NamedAgg(column="clean_name", aggfunc="first")
)

,name
מספר מנפיק,
1395,רימון מניה לא סחירה
1486,סופרגז
1682,אופיסי
1804,סופרגז
1928,רימון
281,אייסיאל
28327,OPC POWER VENTURES LP
576,חברה לישראל
NaN,סופרגז


In [69]:
def holdings_col_mask_by_another_mask(holdings, col_name, mask):
    selected_col_values = holdings.loc[mask, col_name].dropna().unique()
    col_mask = holdings[col_name].isin(selected_col_values)
    print("matching holdings by {}: {}".format(col_name, col_mask.sum()))
    return col_mask

In [70]:
# by issuer num
issuer_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "מספר מנפיק", name_mask)
ISIN_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "ISIN", name_mask)
LEI_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "LEI", name_mask)
il_corp_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "מספר תאגיד", name_mask)
il_sec_num_mask = holdings_col_mask_by_another_mask(all_holdings_cls, 'מספר ני"ע', name_mask)

matching holdings by מספר מנפיק: 3216
matching holdings by ISIN: 2784
matching holdings by LEI: 0
matching holdings by מספר תאגיד: 3317
matching holdings by מספר ני"ע: 3104


In [71]:
all_ids_mask = name_mask | issuer_mask | ISIN_mask | LEI_mask | il_corp_mask | il_sec_num_mask
no_name_match = all_holdings_cls[all_ids_mask & (~name_mask)]
print("selected holdings with no name match: {}".format(no_name_match.shape[0]))

selected holdings with no name match: 234


In [74]:
no_name_match.sort_values("is_fossil", ascending=False)

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,clean_name
303258,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.04,"3,604.36",0.01,0.00,1824549,"אג""ח קונצרני",TASE,NaN,2.39,"3,475,752.20",103.70,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,27/05/2020,2020 רבעון 1,1.00,"3,604.36",IL0057602364,520028010,NaN,חלישראל אג
516755,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.0114,"2,119.54",0.01,0.00,2150933,"אג""ח קונצרני",TASE,NaN,1.68,"1,974,053.22",107.37,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0,nan,nan,02/12/2021,2021 רבעון 3,1.00,"2,119.54",IL0057602364,520028010,NaN,חלישראל אג
260646,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.04,0.01,"5,154.03",0.02,0.00,2014144,"אג""ח קונצרני",TASE,NaN,2.03,"4,815,945.98",107.02,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,24/03/2021,2020 רבעון 4,1.00,"5,154.03",IL0057602364,520028010,NaN,חלישראל אג
149389,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.03,"3,218.08",0.01,0.00,1879029,"אג""ח קונצרני",TASE,NaN,2.47,"3,089,557.51",104.16,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,30/08/2020,2020 רבעון 2,1.00,"3,218.08",IL0057602364,520028010,NaN,חלישראל אג
203528,ISRAE 4.5 12/24,245217,281,ilBBB-,S&P מעלות,דולר אמריקאי,0.04,0.02,"3,658.36",0.00,0.00,1948374,"אג""ח קונצרני",TASE,NaN,3.82,"960,000.00",110.75,0.00,0.00,NaN,Materials,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"מיטב דש גמל ופנסיה בע""מ",I_512065202,0.0,nan,nan,06/12/2020,2020 רבעון 3,1.00,"3,658.36",None,520027830,NaN,ISRAE
306998,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.02,"4,758.65",0.02,0.00,1943203,"אג""ח קונצרני",TASE,NaN,2.24,"4,415,557.51",107.77,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,26/11/2020,2020 רבעון 3,1.00,"4,758.65",IL0057602364,520028010,NaN,חלישראל אג
470334,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.02,"2,600.16",0.01,0.00,2051177,"אג""ח קונצרני",TASE,NaN,1.79,"2,429,603.98",107.02,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,31/05/2021,2021 רבעון 1,1.00,"2,600.16",IL0057602364,520028010,NaN,חלישראל אג
468511,ISRAE 4.5 12/24,None,281,BBB-,S&P,דולר אמריקאי,0.04,0.01,"3,484.24",0.00,0.00,2016501,"אג""ח קונצרני",אחר,NaN,3.65,"960,000.00",112.89,0.00,0.00,בלומברג,Materials,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"מיטב דש גמל ופנסיה בע""מ",I_512065202,0.0,nan,nan,31/03/2021,2020 רבעון 4,1.00,"3,484.24",IL0028102734,520027830,NaN,ISRAE
456335,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.01,"2,115.20",0.01,0.00,2104058,"אג""ח קונצרני",TASE,NaN,1.93,"1,974,053.22",107.15,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,02/09/2021,2021 רבעון 2,1.00,"2,115.20",IL0057602364,520028010,NaN,חלישראל אג
437836,ISHARES S&P GBL CLEAN ENERGY,None,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,16.80,0.00,0.00,1949813,קרנות סל,NYSE,NaN,nan,264.00,"1,849.00",0.00,0.00,NaN,מניות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/12/2020,

In [73]:
all_holdings_cls.loc[all_ids_mask, "is_fossil"].value_counts(dropna=False)

1.00    3193
nan      304
0.00      32
Name: is_fossil, dtype: int64

In [103]:
# change classification
all_holdings_cls.loc[all_ids_mask, "is_fossil"] = 0
all_holdings_cls["שווי פוסילי"] = all_holdings_cls["שווי"] * all_holdings_cls["is_fossil"]

## Misc queries

In [72]:
haifa_q3 = all_holdings_cls.loc[
    (all_holdings_cls["ParentCorpName"].str.contains("חיפה")) &
    (all_holdings_cls["ReportPeriodDesc"] == '2021 רבעון 3')
]

haifa = all_holdings_cls.loc[
    (all_holdings_cls["ParentCorpName"].str.contains("חיפה"))
]

In [73]:
get_summary('ReportPeriodDesc', 'ParentCorpName', holdings=haifa)

,ReportPeriodDesc,ParentCorpName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים,שווי בסוגי החזקות לא פוסיליים,שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים
0,2020 רבעון 1,"קופת""ג של עובדי עירית חיפה","240,715.37","13,120.49",0.05,"108,454.54","13,108.66",0.12,"87,596.72",0.07
1,2020 רבעון 2,"קופת""ג של עובדי עירית חיפה","250,150.81","11,085.24",0.04,"108,377.36","11,072.34",0.10,"85,507.36",0.06
2,2020 רבעון 3,"קופת""ג של עובדי עירית חיפה","255,221.80","7,915.51",0.03,"100,511.10","7,913.27",0.08,"90,400.32",0.04
3,2020 רבעון 4,"קופת""ג של עובדי עירית חיפה","260,795.44","7,505.03",0.03,"107,191.31","7,503.02",0.07,"81,635.06",0.04
4,2021 רבעון 1,"קופת""ג של עובדי עירית חיפה","262,667.78","8,102.91",0.03,"108,852.41","8,102.14",0.07,"77,062.77",0.04
5,2021 רבעון 2,"קופת""ג של עובדי עירית חיפה","263,148.26","8,445.76",0.03,"112,306.53","8,445.66",0.08,"73,023.55",0.05
6,2021 רבעון 3,"קופת""ג של עובדי עירית חיפה","258,155.80","6,478.05",0.03,"95,726.14","6,478.05",0.07,"71,384.13",0.04


In [71]:
haifa_q3[haifa_q3["is_fossil"] == 1]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI
429909,חשמל אגח 29,6000236,600,ilAA+,מעלות S&P,שקל חדש,0.04,-0.0143,799.27,0.02,0.00,2130416,"אג""ח קונצרני",TASE,NaN,3.66,"620,022.00",128.91,0.00,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,799.27,IL0060002362,520000472,NaN
429927,פז נפט אגח ז,1142595,1363,ilAA-,מעלות S&P,שקל חדש,0.01,-0.0056,247.57,0.01,0.00,2130416,"אג""ח קונצרני",TASE,NaN,4.68,"220,908.87",112.07,0.00,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,247.57,IL0011425951,510216054,NaN
429928,פז נפט אגח ו,1139542,1363,ilAA-,מעלות S&P,שקל חדש,0.02,-0.0093,391.21,0.01,0.00,2130416,"אג""ח קונצרני",TASE,NaN,3.64,"340,657.83",114.84,0.00,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,391.21,IL0011395428,510216054,NaN
429939,חברת חשמל אגח 26,6000202,600,ilAA+,מעלות S&P,שקל חדש,0.05,0.0009,564.60,0.02,0.00,2130416,"אג""ח קונצרני",TASE,NaN,1.50,"371,163.42",107.05,167.27,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,564.60,IL0060002024,520000472,NaN
429954,חברהלישראלאגח14,5760301,576,ilA,מעלות S&P,שקל חדש,0.02,0.0191,521.71,0.01,0.00,2130416,"אג""ח קונצרני",TASE,NaN,4.24,"512,331.00",101.83,0.00,0.00,אחר,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,521.71,IL0057603016,520028010,NaN
429980,ישראמקו יהש,232017,232,NaN,NaN,שקל חדש,NaN,NaN,"2,683.44",0.04,0.01,2130416,מניות,TASE,NaN,nan,"3,292,559.94",81.50,0.00,0.00,אחר,חיפושי נפט וגז,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,"2,683.44",IL0002320179,550010003,NaN
429985,ג'נריישן קפיטל,1156926,1769,NaN,NaN,שקל חדש,NaN,NaN,632.92,0.01,0.00,2130416,מניות,TASE,NaN,nan,"664,833.00",95.20,0.00,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,632.92,IL0011569261,515846558,NaN
429986,פז נפט,1100007,1363,NaN,NaN,שקל חדש,NaN,NaN,362.23,0.01,0.00,2130416,מניות,TASE,NaN,nan,"1,032.00","35,100.00",0.00,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,362.23,IL0011000077,510216054,NaN
429991,"דלק ניהול קידוחים יה""ש",475020,475,NaN,NaN,שקל חדש,NaN,NaN,275.10,0.00,0.00,2130416,מניות,TASE,NaN,nan,"48,907.35",562.50,0.00,0.00,אחר,חיפושי נפט וגז,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0,nan,nan,28/10/2021,2021 רבעון 3,1.00,275.10,IL0004750209,550013098,NaN


In [75]:
haifa[haifa["is_fossil"] == 1].sort_values("שם המנפיק/שם נייר ערך")

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI
323136,DELEK(DK US),NaN,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,114.00,0.00,0.00,1816306,מניות,NYSE,NaN,nan,"2,029.00","1,576.00",0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,13/05/2020,2020 רבעון 1,1.00,114.00,US2466471016,520032681,NaN
306831,DEVTAM 4.435 12/30/20,NaN,NaN,Baa3,MOODYS,דולר אמריקאי,0.04,0.03,"2,782.03",0.05,0.01,1859023,"אג""ח קונצרני",TASE,NaN,0.48,"800,000.00",100.33,0.00,0.00,בלומברג,חיפושי נפט וגז,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,28/07/2020,2020 רבעון 2,1.00,"2,782.03",IL0011321663,514914001,NaN
222558,DEVTAM 4.435 12/30/20,NaN,NaN,Baa3,MOODYS,דולר אמריקאי,0.04,0.04,698.17,0.01,0.00,1923377,"אג""ח קונצרני",TASE,NaN,0.24,"200,000.00",101.45,0.00,0.00,בלומברג,חיפושי נפט וגז,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,01/11/2020,2020 רבעון 3,1.00,698.17,IL0011321663,514914001,NaN
322992,DEVTAM 4.435 12/30/20,NaN,NaN,Baa3,MOODYS,דולר אמריקאי,0.04,0.04,"2,879.34",0.05,0.01,1816306,"אג""ח קונצרני",TASE,NaN,0.71,"800,000.00",100.96,0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,13/05/2020,2020 רבעון 1,1.00,"2,879.34",IL0011321663,514914001,NaN
323116,DIAMONDBACK ENERGY INC,NaN,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,28.77,0.00,0.00,1816306,מניות,אחר,NaN,nan,308.00,"2,620.00",0.00,0.00,בלומברג,Other,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,13/05/2020,2020 רבעון 1,1.00,28.77,US25278X1090,NaN,549300R22LSX6OHWEN64
151707,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,434.00,0.01,0.00,2032597,מניות,אחר,NaN,nan,"11,177.00",992.40,0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,28/04/2021,2021 רבעון 1,1.00,434.00,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033
170226,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,399.57,0.01,0.00,1980276,מניות,אחר,NaN,nan,"11,177.00",906.40,0.00,0.00,בלומברג,Other,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,26/01/2021,2020 רבעון 4,1.00,399.57,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033
323117,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,411.57,0.01,0.00,1816306,מניות,אחר,NaN,nan,"11,177.00",944.10,0.00,0.00,בלומברג,Other,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,13/05/2020,2020 רבעון 1,1.00,411.57,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033
222676,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,424.23,0.01,0.00,1923377,מניות,אחר,NaN,nan,"11,177.00",942.80,0.00,0.00,בלומברג,Other,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,01/11/2020,2020 רבעון 3,1.00,424.23,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033
343310,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,422.43,0.01,0.00,2084579,מניות,אחר,NaN,nan,"11,177.00",975.40,0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,28/07/2021,2021 רבעון 2,1.00,422.43,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033


## looking into specific holdings
קרנות סל - מגדל 125

In [549]:
all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל') &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("125")) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("מגדל"))
].groupby('מספר ני"ע')["שווי"].sum()

מספר ני"ע
1150283   89,073.06
1159714   37,317.87
1174796       17.59
Name: שווי, dtype: float64

In [569]:
# החזקות של חברות בקרן עוקבת תא 125 של מגדל
cols = ["SystemName", "ParentCorpName","שווי", 'שעור מנכסי אפיק ההשקעה', "שעור מערך נקוב מונפק"]
all_holdings_2021q2[
    (all_holdings_2021q2['מספר ני"ע'] == 1150283)
][cols]

,SystemName,ParentCorpName,שווי,שעור מנכסי אפיק ההשקעה,שעור מערך נקוב מונפק
13831,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",231.34,0.00,0.00
18897,פנסיה,"מיטב דש גמל ופנסיה בע""מ","16,149.28",0.01,0.01
29346,ביטוח,"איילון חברה לביטוח בע""מ",560.67,0.00,0.00
40372,גמל,"פסגות קופות גמל ופנסיה בע""מ","16,325.29",0.01,0.02
74680,גמל,"הפניקס פנסיה וגמל בע""מ","1,050.54",0.00,0.00
122078,גמל,"מיטב דש גמל ופנסיה בע""מ","25,739.15",0.01,0.02
160766,גמל,"החברה לניהול קרן ההשתלמות להנדסאים וטכנאים בע""מ","1,266.51",0.00,0.00
248891,פנסיה,"פסגות קופות גמל ופנסיה בע""מ","9,443.80",0.03,0.01
279942,גמל,"אינפיניטי ניהול השתלמות וגמל בע""מ","3,342.95",0.01,0.00
398997,גמל,"החברה לניהול קרן השתלמות למשפטנים בע""מ",134.76,0.00,0.00


In [575]:
cols = ["שם המנפיק/שם נייר ערך", 'מספר ני"ע', "שווי", "שעור מנכסי אפיק ההשקעה", "שעור מערך נקוב מונפק", "SystemName"]

all_holdings_2021q2[
    (all_holdings_2021q2['holding_type'] == 'קרנות סל') &
    (all_holdings_2021q2["ParentCorpName"].str.contains('מגדל')) &
    (~all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('כש')) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('125'))
][cols]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",שווי,שעור מנכסי אפיק ההשקעה,שעור מערך נקוב מונפק,SystemName
88639,הראל סל תא 125,1148899,"77,115.37",0.01,0.09,ביטוח
88644,פסגות ETF תל אביב 125,1148808,"155,533.51",0.01,0.12,ביטוח
88648,קסם תא125,1146356,"140,897.26",0.01,0.07,ביטוח
88650,תכלית תא 125,1143718,"47,261.98",0.00,0.02,ביטוח
281587,"הראל סל תא 125- הראל קרנות נאמנות בע""מ",1148899,"14,195.15",0.01,0.02,גמל
281591,"*MTF סל )4A( ת""א125- אקלים נקי- מגדל קרנות נאמ...",1174796,17.59,0,0,גמל
281596,"תכלית סל תא 125- מיטב תכלית קרנות נאמנות בע""מ",1143718,"9,595.19",0.00,0.00,גמל
281599,"פסגות ETF תא 125- פסגות קרנות מדדים בע""מ",1148808,"27,112.11",0.01,0.02,גמל
281606,"קסם ETF תא 125- קסם קרנות נאמנות בע""מ",1146356,"24,544.03",0.01,0.01,גמל
455627,הראל סל תא 125,1148899,"56,424.02",0.01,0.06,פנסיה


## Looking into ETFs

In [557]:
etf_holdings = all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל')
]

etf_holdings.groupby("זירת מסחר")["שווי"].sum()

זירת מסחר
AMEX              118.70
ASX           483,289.25
CAC           530,710.30
DAX           121,456.62
EURONEXT    4,150,375.49
FWB         2,293,354.79
HKSE        1,230,909.60
ISE            28,392.81
JPX           187,602.86
LSE        15,767,218.60
NASDAQ      9,568,288.10
NYSE       39,434,670.96
SIX         1,638,552.83
TASE       30,436,824.11
TSE         4,318,613.35
TSX            26,454.18
אחר        13,829,003.07
Name: שווי, dtype: float64

In [562]:
etf_holdings_tase = etf_holdings[etf_holdings["זירת מסחר"] == "TASE"]
etf_holdings_tase.head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc
8585,MTF סל S&P 500 מנוטר,1150572,511303661,NaN,NaN,שקל חדש,NaN,NaN,"1,372.16",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,23847,5754,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8586,MTF סל indx USA clou,1158328,511303661,NaN,NaN,שקל חדש,NaN,NaN,209.26,0.00,0.00,2098344,קרנות סל,TASE,NaN,NaN,3935,5318,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8587,הראל סל NDX 100 מנוט,1149103,511776783,NaN,NaN,שקל חדש,NaN,NaN,"1,439.72",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,10328,13940,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8588,הראל סל SP500 מנוטרל,1149137,511776783,NaN,NaN,שקל חדש,NaN,NaN,"1,727.06",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,38793,4452,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8589,מו.NDX100ממ,1165844,514884485,NaN,NaN,שקל חדש,NaN,NaN,"1,466.68",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,19863,7384,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
